In [ ]:
import numpy as np

from Polyhedra import *
from Distances import *
from Algorithm import *

grid = np.array([
    [2, 0, 0, 0],
    [1, 1, 0, 0],
    [1, 0, 1, 0],
    [1, 0, 0, 1]
], dtype = float)

grid1 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1]
], dtype = float)

grid2 = np.array([
    [1, -1, 0, 0],
    [0, 1, -1, 0],
    [0, 0, 1, -1],
    [0.5, 0.5, 0.5, 0.5]
], dtype = float)

grid3 = np.array([
    [1, 1, 0, 0],
    [1, -1, 0, 0],
    [0, 1, -1, 0],
    [0, 0, 1, -1]
], dtype = float)

grid4 = np.array([
    [1, 0, 0, 0],
    [0, 1, 0, 0],
    [0, 0, 1, 0],
    [0.5, 0.5, 0.5, 0.5]
], dtype = float)

grid5 = np.array([
    [2 ** 0.5, 0., 0., 0.],
    [-1/(2 ** 0.5), 6**0.5/3, 0., 0.],
    [0., -(2/3)**0.5, 2/((3**0.5)*10), 0.],
    [0., 0., -(3**0.5)/2, (5**0.5)/20]
], dtype = float)

In [ ]:
import itertools
from tqdm import tqdm
from math import comb

def canonical_form_by_rows(matrix: np.ndarray) -> np.ndarray:
    """
    Приводит матрицу к каноническому виду через сортировку строк.
    
    matrix — исходная матрица (np.ndarray)
    Возвращает: новая матрица с отсортированными строками
    """

    # Сортируем строки 
    idx = np.lexsort(matrix.T[::-1])  # Инвертируем столбцы для нужного порядка
    return matrix[idx]

# Список матриц решетки grid
list_coords = []

for var in itertools.product(range(-1, 2), repeat=4): 
    # меньше 3 нельзя - тк при поиске центр региона отбрасываем незаконченные регионы
    #if var
    list_coords.append(var)
    
list_coords.remove((1, 0, 0, 0))
    
print(len(list_coords), list_coords[12])

result = []
for lst in list_coords:
    first_non_zero = next((x for x in lst if x != 0), None)
    if first_non_zero is not None and first_non_zero > 0:
        result.append(lst)

print(len(result))

#result = [np.random.rand(4) for _ in range(10)]  # список векторов
list_grids = []  # список уникальных базисов
c = 0  # счётчик хороших матриц
k = 0  # общий счётчик

# Общее количество комбинаций (для корректного tqdm)
n = len(result)
total_combinations = comb(n, 3)

print(f"Всего комбинаций: {total_combinations}")

# Основной цикл с прогресс-баром
for vectors in tqdm(combinations(result, 3), total=total_combinations, desc="Обработка комбинаций"):
    g = np.vstack(([1, 0, 0, 0], *vectors))  # Собираем векторы в матрицу 4x4
    k += 1
    
    if not np.isclose(np.linalg.det(g), 0):  # Если матрица невырожденная
        # Проверяем уникальность (без учёта транспонирования)
        #k += 1
        grid_int = (g).astype(int)
        basis = LLL.reduction(IntegerMatrix.from_matrix(grid_int.tolist()))
        rows, cols = basis.nrows, basis.ncols
        #sub_grid_LLL = np.array([[basis[i, j] for j in range(cols)] for i in range(rows)])
        lll_grid = canonical_form_by_rows(np.array([[basis[i, j] for j in range(cols)] for i \
                                                    in range(rows)]))
        
        if not any(np.allclose(lll_grid, m) for m in list_grids):
        #if not any(np.allclose(g.T, m) for m in list_grids):
            c += 1
            list_grids.append(lll_grid)

In [ ]:
for grid in list_grids:
    vor4 = VoronoiPolyhedra(grid)
    vor4.build()

    det_dist, det_center, det_mat = find_optimal(range(48, 49), 2, grid, vor4, vor4.max_len)

    print(det_dist, det_center, det_mat)

In [ ]:
#grid[0][0] = 2.15

print(grid)

grid_int = (grid).astype(int)
basis = LLL.reduction(IntegerMatrix.from_matrix(grid_int.tolist()))
rows, cols = basis.nrows, basis.ncols
grid = np.array([[basis[i, j] for j in range(cols)] for i in range(rows)])

vor4 = VoronoiPolyhedra(grid)
vor4.build()

det_dist, det_center, det_mat = find_optimal(range(49, 50), 2, grid, vor4, vor4.max_len)

print(det_dist, det_center, det_mat)

In [ ]:

print(grid5)


vor4 = VoronoiPolyhedra(grid5)
vor4.build()

det_dist, det_center, det_mat = find_optimal(range(50, 60), 2, grid5, vor4, vor4.max_len)

print(det_dist, det_center, det_mat)
